## Load multiple models

In [1]:
from local_llm.local_llm import LocalLlm
from local_llm.constants import ModelEnum

instance = LocalLlm([ModelEnum.Llama3_8B_Q6, ModelEnum.Qwen3_8B_Q8])

Loading model Llama3_8B_Q6 from models/Llama3_8B_Q6/model.gguf.


llama_context: n_ctx_per_seq (512) < n_ctx_train (10240) -- the full capacity of the model will not be utilized


Loading model Qwen3_8B_Q8 from models/Qwen3_8B_Q8/model.gguf.


llama_context: n_ctx_per_seq (512) < n_ctx_train (40960) -- the full capacity of the model will not be utilized


## Ask them to analyze statements in order to find argumentative fallacies

In [2]:
question = "Les vaccins sont dangereux car mon voisin a eu des effets secondaires après sa vaccination. De plus, les laboratoires pharmaceutiques ne cherchent qu'à faire du profit. Par conséquent, nous ne devrions pas faire confiance aux vaccins."
#question = "Vaccines are dangerous because my neighbor experienced side effects after his vaccination. Furthermore, pharmaceutical companies are only interested in making a profit. Therefore, we should not trust vaccines."
#question = "Si l'IA devient plus intelligente que les humains, alors soit elle nous aidera, soit elle nous remplacera. Or, une IA superintelligente n'aura pas besoin de nous aider. Donc elle nous remplacera."
#question = "Votre argument sur le climat est invalide car vous n'êtes pas climatologue."
#question = "Si nous n'agissons pas maintenant contre le changement climatique, nos enfants vivront dans un monde invivable. Nous devons donc interdire immédiatement toutes les voitures."
#question = "Tous les cygnes que j'ai vus sont blancs, donc tous les cygnes sont blancs."
#question = "Terrorism is the best political weapon for nothing drives people harder than a fear of sudden death."

print(instance(ModelEnum.Llama3_8B_Q6, question, max_tokens=500))

Prompting...


1. The main arguments are:
- Personal anecdote: The author's neighbor had adverse effects after vaccination.
- Alleged motivation: Pharmaceutical companies only care about profits.

2. These arguments are invalid. 
- The personal anecdote is a classic case of anecdotal evidence fallacy. The author generalizes from one individual's experience to a conclusion about the entire class of vaccines, which is unjustified. Many people receive vaccines without adverse effects.
- The alleged motivation is an ad hominem fallacy. Even if pharmaceutical companies are profit-driven, this does not necessarily make their vaccines unsafe. Safety and efficacy are separate issues. 

3. Therefore, the conclusion that we should not trust vaccines is unwarranted by the provided arguments. We need scientifically sound evidence and not personal accounts or unfounded assumptions.


